In [2]:
from sentence_transformers import SentenceTransformer, util
import numpy as np
import pandas as pd
embedder = SentenceTransformer('bert-base-chinese')

Exception when trying to download https://sbert.net/models/bert-base-chinese.zip. Response 404


In [ ]:
df = pd.read_csv('.//data//Result_TFIDF.csv', index_col=0)
df.head(2)

In [ ]:
test = pd.read_csv('.//data//Result_Test.csv', index_col=0)
test.head(2)

In [ ]:
df1 = df[~df['no'].isin(test['no'])]
df1.head(2)

In [ ]:
import pandas as pd
import time
tStart = time.time()

In [ ]:
class Data(object):
    def __init__(self):
        self.no = "0"
        self.reason = ""
        self.Situation = ""

In [ ]:
#embedder = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

# Corpus with example sentences
#corpus = ['A man is eating food.',
#          'A man is eating a piece of bread.',
#          'The girl is carrying a baby.',
#          'A man is riding a horse.',
#          'A woman is playing violin.',
#          'Two men pushed carts through the woods.',
#          'A man is riding a white horse on an enclosed ground.',
#          'A monkey is playing drums.',
#          'A cheetah is running behind its prey.'
#          ]
corpus = []
corpusList = []
for index, row in df1.iterrows():
    if (row.Situation == None):
        continue
    if (isinstance(row.Situation, float)):
        continue
    if (len(row.Situation) == 0):
        continue
    d = Data()
    d.no = row.no
    d.reason = row.reason
    d.Situation = row.Situation
    corpusList.append(d)
    corpus.append(row.Situation)
corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)
print(len(corpus))

In [ ]:
tEnd = time.time()#計時結束
#列印結果
print("轉向量時間", "%f sec" % (tEnd - tStart))

In [ ]:
# Query sentences:
#queries = ['A man is eating pasta.', 'Someone in a gorilla costume is playing a set of drums.', 'A cheetah chases prey on across a field.']
queries = []
queries.append('吳家軒以手勒住吳偉維脖子，並將他頭部推向福利社遮雨棚之鋼架，再將他摔倒並壓制在地，導致他頭部挫傷、前額紅腫擦傷、右膝及左足挫擦傷等傷害。')
for index, row in test.iterrows():
    queries.append(str(row.Situation))
print(len(queries))

In [ ]:
# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
top_k = 5
i = 0
for query in queries:
    tStart2 = time.time()
    query_embedding = embedder.encode(query, convert_to_tensor=True)
    cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
    cos_scores = cos_scores.cpu()


    #We use np.argpartition, to only partially sort the top_k results
    top_results = np.argpartition(-cos_scores, range(top_k))[0:top_k]
    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")

    no = 1
    for idx in top_results[0:top_k]:
        print(no, "  "
              , corpusList[idx].no,"\t"
              ,  corpusList[idx].reason
              , "\t(Score: %.4f)\n" % (cos_scores[idx])
              , corpus[idx].strip())
        no = no + 1

    tEnd2 = time.time()#計時結束
    print("COS相似計算時間", "%f sec" % (tEnd2 - tStart2))

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
embedder = SentenceTransformer('bert-base-chinese')
corpus_embeddings = embedder.encode(corpus)
# Then, we perform k-means clustering using sklearn:
from sklearn.cluster import KMeans

num_clusters = 18
clustering_model = KMeans(n_clusters=num_clusters)
clustering_model.fit(corpus_embeddings)
cluster_assignment = clustering_model.labels_

clustered_sentences = [[] for i in range(num_clusters)]
for sentence_id, cluster_id in enumerate(cluster_assignment):
    clustered_sentences[cluster_id].append(corpus[sentence_id])

for i, cluster in enumerate(clustered_sentences):
    print("Cluster ", i+1)
    print(cluster)
    print("")
    break